In [ ]:
# !pip install pydub
# !pip install git+https://github.com/openai/whisper.git
# !pip install srt
# !pip install -U yt-dlp
# !pip install   pyannote.audio
# !pip install huggingface-hub
# !pip install -U webvtt-py
# !pip install -q onnxruntime

In [ ]:
# !echo y| conda install -c conda-forge ffmpeg

# 各種パラメーター設定

In [3]:
target_id="shogi"
spacermilli = 0
rawdata_folder_path = ""
analysis_folder_path = ""
rawdata_path = rawdata_folder_path+target_id+".wav"
save_text_path = analysis_folder_path+target_id+"_音声認識_whisper_with_vad.txt"
save_text_path0 = analysis_folder_path+target_id+"speaker0_音声認識_whisper_with_vad.txt"
save_text_path1 = analysis_folder_path+target_id+"speaker1_音声認識_whisper_with_vad.txt"
save_srt_path = analysis_folder_path+target_id+"_音声認識_whisper_with_vad.srt"
#人の声が入っているかどうかを識別するパラメーター。　オーディオチャンク毎に確率が出る。デフォルトは0.5
vad_threshold=0.4
# 音声ファイルを分割するための閾値。大きくすればするほど、一つのファイルに複数の音声が入る。
#whisperの処理性能的にある程度長めの音声で判定しているので、一つのファイルに複数の音声を詰め込むと処理性能が落ちる？
#要実験
chunk_threshold = 2
#音声に対する感度を少し下げる（このワードを検知したら無視する可能性が高い）
suppress_low = [
    "視聴",
    "ご視聴",
]
#音声に対する感度を大きく下げる
suppress_high = [
    "ご視聴",
    "視聴ありがとうございました",
    "ご視聴ありがとうございました!"
    "私はあなたを愛しています。"
]
#音声に対する感度を大きく上げる
promotion_high = [,]
#whisperの試行回数
max_attempts=1
#whisperモデルで無音判定する閾値。高いほど1.0に近づくほど無音の可能性が高い。デフォルトは0.6
no_speech_threshold= 0.7
#whisperモデルで喋り声の判定する閾値。高いほど喋っている可能性が高い。デフォルト-1.0
avg_logprob = -1.0
# vad(voice acitivity detectorモデルのインストール)
SAMPLING_RATE = 16000
speakers_num=2

# インポート

In [5]:
from srt import Subtitle
from pydub import AudioSegment
import srt
import whisper
from IPython.display import Audio
from pprint import pprint
from tqdm import tqdm
import datetime
import whisper
import os
import re
from datetime import timedelta
import torch
from pyannote.audio import Pipeline

In [6]:
import torch
print(torch.cuda.is_available())

True


In [11]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (!aws).
Your token has been saved to /home/ec2-user/.huggingface/token
Login successful


In [8]:
torch.set_num_threads(1)

USE_ONNX = False 
model_vad, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                                  model='silero_vad',
                                  force_reload=True,
                                  onnx=USE_ONNX)

(get_speech_timestamps,
 save_audio,
 read_audio,
 VADIterator,
 collect_chunks) = utils    

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /home/ec2-user/.cache/torch/hub/master.zip


# pyannoteによる話者分離

In [ ]:
pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization', use_auth_token=True)

In [13]:
dz = pipeline(rawdata_path, num_speakers=speakers_num)  

In [14]:
with open(target_id+"_diarization.txt", "w") as text_file:
    text_file.write(str(dz))

In [15]:
def millisec(timeStr):
  spl = timeStr.split(":")
  s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
  return s

# vadによる話し声の有無判定

In [17]:
#タイムスタンプの作成
sound_val = read_audio(rawdata_path, sampling_rate=SAMPLING_RATE)
speech_timestamps = get_speech_timestamps(sound_val, model_vad, sampling_rate=SAMPLING_RATE, threshold=vad_threshold)
#pprint(speech_timestamps)

In [18]:
# 声の間がchunk_thresholdよりも長かったら音声ファイルを複数に分割する様にする。
u = [[]]
for i in range(len(speech_timestamps)):
    if i > 0 and speech_timestamps[i]["start"] > speech_timestamps[i - 1]["end"] + (chunk_threshold * SAMPLING_RATE):
        u.append([])
    u[-1].append(speech_timestamps[i])

In [19]:
# 人の声が入っている部分だけを抽出。
#処理時間を短縮することができる
for i in range(len(u)):
    save_audio("vad_chunks_" + str(i) + ".wav",
               collect_chunks(u[i], sound_val),
               sampling_rate=SAMPLING_RATE,)

In [20]:
# タイムスタンプを秒に変換
for i in range(len(u)):
    time = 0.0
    offset = 0.0
    for j in range(len(u[i])):
        u[i][j]["start"] /= SAMPLING_RATE
        u[i][j]["end"] /= SAMPLING_RATE
        u[i][j]["chunk_start"] = time
        time += u[i][j]["end"] - u[i][j]["start"]
        u[i][j]["chunk_end"] = time
        if j == 0:
            offset += u[i][j]["start"]
        else:
            offset += u[i][j]["start"] - u[i][j - 1]["end"]
        u[i][j]["offset"] = offset

# whisperによる音声解析

In [21]:
mode_whisperl = whisper.load_model("large")

100%|█████████████████████████████████████| 2.87G/2.87G [00:44<00:00, 69.9MiB/s]


In [ ]:
test_counter =0
subs = []
subs_text = []
segment_info = []
sub_index = 1
for i in tqdm(range(len(u))):
    line_buffer = []  # Used for DeepL
    for x in range(max_attempts):
        result = mode_whisperl.transcribe(
            "vad_chunks_" + str(i) + ".wav", 
            verbose=False, language="ja",
            no_speech_threshold=no_speech_threshold,temperature=0.2,
        logprob_threshold=avg_logprob)
        # Break if result doesn't end with severe hallucinations
        if len(result["segments"]) == 0:
            break
        elif result["segments"][-1]["end"] < u[i][-1]["chunk_end"] + 10.0:
            break
        elif x+1 < max_attempts:
            print("Retrying chunk", i)
    for r in result["segments"]:
        # Skip audio timestamped after the chunk has ended
        if r["start"] > u[i][-1]["chunk_end"]:
            continue
        # 特定のワード、フレーズに対して感度を調整
        for s in suppress_low:
            if s in r["text"]:
                r["avg_logprob"] -= 0.30
        for s in suppress_high:
            if s in r["text"]:
                r["avg_logprob"] -= 0.50
        for s in promotion_high:
            if s in r["text"]:
                r["avg_logprob"] += 0.20
         # Keep segment info for debugging
        del r["tokens"]
        segment_info.append(r)
        if r["avg_logprob"] < avg_logprob or r["no_speech_prob"] > no_speech_threshold:
            #print("do_skip")
            continue
        # セグメント情報を保持
        segment_info.append(r)
        # Set start timestamp
        start = r["start"] + u[i][0]["offset"]
        for j in range(len(u[i])):
            if (
                r["start"] >= u[i][j]["chunk_start"]
                and r["start"] <= u[i][j]["chunk_end"]
            ):
                start = r["start"] + u[i][j]["offset"]
                break
        # subsの重複を避ける
        if len(subs) > 0:
            last_end = datetime.timedelta.total_seconds(subs[-1].end)
            if last_end > start:
                subs[-1].end = datetime.timedelta(seconds=start)
        # タイムスタンプの終わりを判定
        end = u[i][-1]["end"] + 0.5
        for j in range(len(u[i])):
            if r["end"] >= u[i][j]["chunk_start"] and r["end"] <= u[i][j]["chunk_end"]:
                end = r["end"] + u[i][j]["offset"]
                break
        # Add to SRT list
        subs.append(
            srt.Subtitle(
                index=sub_index,
                start=datetime.timedelta(seconds=start),
                end=datetime.timedelta(seconds=end),
                content=r["text"].strip(),
            )
        )
        subs_text.append(r["text"].strip())
        sub_index += 1

100%|██████████| 1/1 [04:57<00:00, 297.54s/it]


# whisperとpyaanoteの統合

In [49]:
#話者分類
speaker_zero = []
speaker_one= []
counter=0
#話はじめがpyaanoteにより分類された話時間の内のどちらに含まれるかで分類
#厳密にやるならwhisper文字起こしの解析結果がpyaanoteで解析された話時間のどちらが長いか判定すれば良い
for i in range(len(dzs)):
    end_time = datetime.timedelta(hours= float(dzs[i][20:22]),
                   minutes=float(dzs[i][23:25]),seconds=float(dzs[i][26:32]))
    try:
        if dzs[i][-10:]=="SPEAKER_00":
            while subs[counter].start < end_time:
                speaker_zero.append(subs[counter].content.strip())
                counter += 1
                #print(i,counter,"SPEAKER_00")
        if dzs[i][-10:]=="SPEAKER_01":
            while subs[counter].start < end_time:
                speaker_one.append(subs[counter].content.strip())
                counter += 1
    except IndexError:
        print(counter)
        print('処理が完了しました。')

471
処理が完了しました。
471
処理が完了しました。


In [34]:
#srtファイルの作成
with open(save_srt_path, "w", encoding="utf8") as f:
    f.write(srt.compose(subs))
#textファイルの作成
with open(save_text_path, "w", encoding="utf8") as f:
    f.write(''.join(subs_text))

In [35]:
# pyaanoteによる話者分離結果を記載
with open("diarization.txt", "w") as text_file:
    text_file.write(str(dz))
#話者1のファイルの作成
with open(save_text_path0, "w", encoding="utf8") as f:
    f.write(''.join(speaker_zero))
#話者1のファイルの作成    
with open(save_text_path1, "w", encoding="utf8") as f:
    f.write(''.join(speaker_one))

In [ ]:
try:
    for i in range(len(u)):
        os.remove("vad_chunks_" + str(i) + ".wav")
except:
     print("error")

# htmlファイルの作成

In [76]:
preS = '<!DOCTYPE html><html lang="ja">'
postS = '\t</body>\n</html>'

In [ ]:
import re
dzs = open('diarization.txt').read().splitlines()

groups = []
g = []
lastend = 0

for d in dzs:   
  if g and (g[0].split()[-1] != d.split()[-1]):      #same speaker
    groups.append(g)
    g = []
  
  g.append(d)
  
  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
  end = millisec(end)
  if (lastend > end):       #segment engulfed by a previous segment
    groups.append(g)
    g = [] 
  else:
    lastend = end
if g:
  groups.append(g)
print(*groups, sep='\n')

In [78]:
speakers = {'SPEAKER_00':('speaker0', '#CFF5E7', 'mint'), 'SPEAKER_01':('speaker1', '#A0E4CB', 'teal') ,
           }

In [79]:
html = list(preS)
gidx = 0
sapcer_datetime = datetime.timedelta(hours= 0,minutes=0,seconds=spacermilli/1000)
for g in groups:  
    shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[1]
    end_time =datetime.timedelta(hours= float(shift[0:2]),minutes=float(shift[3:5]),seconds=float(shift[6:])) - sapcer_datetime #the start time in the original video
    if subs[gidx]:
        speaker = g[0].split()[-1]
    if speaker in speakers:
        speaker, boxclr, spkrclr = speakers[speaker] 
    if subs[gidx].start < end_time:
        html.append(f'<div class="e" style="background-color: {boxclr}">\n');
        html.append(f'<span style="color: {spkrclr}">{speaker}　　　　{subs[gidx].start}</span><br>\n')
    try:
        while subs[gidx].start < end_time:
          html.append(f'<div class="c">')
          html.append(f'\t\t\t\t<div class="t"> {subs[gidx].content}</div><br>\n')
          html.append(f'</div>')
          #print(speaker, boxclr, spkrclr,gidx,subs[gidx].content)
          gidx += 1
        html.append(f'</div>\n');
    except IndexError:
        html.append(f'</div>\n');
        break

html.append(postS)
s = "".join(html)

with open(target_id+"_capspeaker.html", "w") as text_file:
    text_file.write(s)